### hope to speed up existing rgf by using memory speed ups

### helper functions 

In [34]:
import numpy as np
class Helper_functions:
    
    # Define F_1/2(x)
    def FD_half(x):
        '''
        Approximation of the Fermi-Dirac integral of order 1/2.
        Reference: http://dx.doi.org/10.1063/1.4825209
        '''
        v = x**4 + 50 + 33.6 * x * (1 - 0.68 * np.exp(-0.17 * (x + 1)**2))
        return 1 / (np.exp(-x) + 3 * np.pi**0.5 / 4 * v**(-3/8))

    # Define F_-1/2(x) as the derivative of F_1/2(x)
    def FD_minus_half(x):
        dx = x * 1e-6  
        return (Helper_functions.FD_half(x + dx) - Helper_functions.FD_half(x - dx)) / (2 * dx)

    def sparse_diag_product(A, B):
        """
        Compute diagonal elements of C = A * B efficiently for sparse matrices.

        Parameters:
            A (csr_matrix): sparse matrix in CSR format
            B (csc_matrix): sparse matrix in CSC format

        Returns:
            numpy.ndarray: diagonal elements of A*B
        """
        from scipy.sparse import csr_matrix, csc_matrix
        import numpy as np

        # Ensure A is CSR and B is CSC for efficient indexing
        if not isinstance(A, csr_matrix):
            A = csr_matrix(A)
        if not isinstance(B, csc_matrix):
            B = csc_matrix(B)

        n = A.shape[0]
        diag = np.zeros(n, dtype=complex)

        for i in range(n):
            # Get row i from A (CSR format)
            A_row_start, A_row_end = A.indptr[i], A.indptr[i+1]
            A_cols = A.indices[A_row_start:A_row_end]
            A_vals = A.data[A_row_start:A_row_end]

            # Get column i from B (CSC format)
    
            B_rows = B.indices[B.indptr[i]:B.indptr[i+1]]
            B_vals = B.data[B.indptr[i]:B.indptr[i+1]]

            # Compute intersection of indices efficiently
            ptr_a, ptr_b = 0, 0
            sum_diagonal = 0.0
            while ptr_a < len(A_cols) and ptr_b < len(B_rows):
                col_a, row_b = A_cols[ptr_a], B_rows[ptr_b]
                if col_a == row_b:
                    sum_diagonal += A_vals[ptr_a] * B_vals[ptr_b]
                    ptr_a += 1
                    ptr_b += 1
                elif col_a < row_b:
                    ptr_a += 1
                else:
                    ptr_b += 1

            diag[i] = sum_diagonal

        return diag



### benchmark

In [ ]:
"""
speed_rgf_vs_inverse.py
-----------------------
Benchmarks a stripped-down recursive Green-function (RGF) implementation
against a brute-force matrix inverse on a block-tridiagonal tight-binding
chain with two self-energy contacts.

Python ≥ 3.9, NumPy ≥ 1.24
"""

import time
import numpy as np

# ----------------------------------------------------------------------
# 1.  Utility: create a random Hermitian block-tridiagonal Hamiltonian
# ----------------------------------------------------------------------
def build_test_system(num_blocks: int, block_size: int, seed: int = 1234):
    """
    Returns
        H                (N × N Hermitian),  N = num_blocks*block_size
        Sigma_left       (N × N)    – contact self-energy on first block
        Sigma_right      (N × N)    – contact self-energy on last  block
    """
    rng = np.random.default_rng(seed)
    bs = block_size
    N  = num_blocks * bs

    # diagonal and off-diagonal blocks
    H_diag = rng.normal(size=(num_blocks, bs, bs)) + 1j * rng.normal(size=(num_blocks, bs, bs))
    H_diag = 0.5 * (H_diag + np.conjugate(np.swapaxes(H_diag, -1, -2)))  # Hermitianise

    H_off  = rng.normal(size=(num_blocks-1, bs, bs)) + 1j * rng.normal(size=(num_blocks-1, bs, bs))
    
    # build full matrix
    H = np.zeros((N, N), dtype=np.complex128)
    for i in range(num_blocks):
        s, e = i*bs, (i+1)*bs
        H[s:e, s:e] = H_diag[i]
        if i < num_blocks-1:
            H[s:e, e:e+bs] = H_off[i]
            H[e:e+bs, s:e] = H_off[i].conj().T
    
    # simple wide-band self-energies on the first/last block
    Gamma_cpl = 0.05   # coupling strength
    Sigma_left  = np.zeros_like(H)
    Sigma_right = np.zeros_like(H)
    Sigma_left[ :bs,  :bs]  = -0.5j * Gamma_cpl * np.eye(bs)
    Sigma_right[-bs:, -bs:] = -0.5j * Gamma_cpl * np.eye(bs)
    
    return H, N, Sigma_left, Sigma_right, block_size

# ----------------------------------------------------------------------
# 2.  Recursive Green-function (diagonal only) – retarded & lesser
# ----------------------------------------------------------------------
dagger = lambda A: np.conjugate(A).T
def rgf_retarded_lesser(E, H, N,
                        self_energy_left, self_energy_right,
                        f_s, f_d, block_size):

    # 0) pre-compute once
    gamma1 = 1j * (self_energy_left  - dagger(self_energy_left))
    gamma2 = 1j * (self_energy_right - dagger(self_energy_right))
    self_energy_lesser = gamma1 * f_s + gamma2 * f_d

    num_blocks  = N // block_size
    E_matrix    = np.eye(N, dtype=complex) * E
    A           = E_matrix - H - self_energy_left - self_energy_right
    I_blk       = np.eye(block_size, dtype=complex)

    # 1) contiguous storage instead of Python lists
    g_R_blocks       = np.empty((num_blocks, block_size, block_size), dtype=complex)
    g_lesser_blocks  = np.empty_like(g_R_blocks)

    G_R      = [None] * num_blocks
    G_R_1    = [None] * (num_blocks - 1)
    G_lesser = [None] * num_blocks
    G_lesser_1 = [None] * (num_blocks - 1)

    # ------------------------------------------------------------------
    # forward sweep
    # ------------------------------------------------------------------
    for i in range(num_blocks):
        start, end   = i * block_size, (i + 1) * block_size
        prev         = (i - 1) * block_size

        if i == 0:                                   # first block
            g_0_r = np.linalg.solve(A[start:end, start:end], I_blk)
            g_R_blocks[0] = g_0_r
            g_lesser_blocks[0] = g_0_r @ self_energy_lesser[start:end, start:end] @ dagger(g_0_r)
        else:
            H_eff = (
                A[start:end, start:end]
                - A[start:end, prev:start]
                  @ g_R_blocks[i - 1]
                  @ A[prev:start, start:end]
            )
            g_i_r = np.linalg.solve(H_eff, I_blk)
            g_R_blocks[i] = g_i_r

            sigma_lesser = (
                A[start:end, prev:start]
                @ g_lesser_blocks[i - 1]
                @ dagger(A[prev:start, start:end])
            )
            g_i_lesser = g_i_r @ (
                self_energy_lesser[start:end, start:end]
                + sigma_lesser
                - self_energy_lesser[start:end, prev:start]
                  @ dagger(g_R_blocks[i - 1])
                  @ dagger(A[prev:start, start:end])
                - A[start:end, prev:start]
                  @ g_R_blocks[i - 1]
                  @ self_energy_lesser[prev:start, start:end]
            ) @ dagger(g_i_r)

            g_lesser_blocks[i] = g_i_lesser

    G_R[-1]      = g_R_blocks[-1]
    G_lesser[-1] = g_lesser_blocks[-1]

    # ------------------------------------------------------------------
    # backward sweep
    # ------------------------------------------------------------------
    for i in reversed(range(num_blocks - 1)):
        start, end   = i * block_size, (i + 1) * block_size
        after        = (i + 2) * block_size

        # retarded
        G_R[i] = g_R_blocks[i] @ (
            I_blk
            + A[start:end, end:after] @ G_R[i + 1] @ A[after - block_size:after, start:end] @ g_R_blocks[i]
        )
        G_R_1[i] = -G_R[i + 1] @ A[after - block_size:after, start:end] @ g_R_blocks[i]

        # --- lesser ----------------------------------------------------
        gr0 = g_R_blocks[i]            # <-- re-use, no new inverse
        ga0 = dagger(gr0)
        gr1 = g_R_blocks[i + 1]        # <-- re-use
        ga1 = dagger(gr1)

        gqq1 = gr0 @ self_energy_lesser[start:end, end:after]   @ ga1
        gq1q = gr1 @ self_energy_lesser[end:after, start:end]   @ ga0

        G_i_lesser = (
            g_lesser_blocks[i]
            + g_R_blocks[i]
              @ (A[start:end, end:after] @ G_lesser[i + 1] @ dagger(A[end:after, start:end]))
              @ dagger(g_R_blocks[i])
            - (g_lesser_blocks[i] @ A[end:after, start:end] @ dagger(G_R_1[i].T)
               + G_R_1[i].T @ A[end:after, start:end] @ g_lesser_blocks[i])
            - (gqq1 @ dagger(A[end:after, start:end]) @ dagger(G_R[i])
               + G_R[i] @ A[start:end, end:after]     @ gq1q)
        )
        G_lesser[i] = G_i_lesser

        G_i_lesser_1 = (
            gq1q
            - G_R_1[i] @ A[start:end, end:after] @ gq1q
            - G_R[i + 1] @ A[end:after, start:end] @ g_lesser_blocks[i]
            - G_lesser[i + 1] @ dagger(A[end:after, start:end]) @ dagger(g_R_blocks[i])
        )
        G_lesser_1[i] = G_i_lesser_1[0]

    # ------------------------------------------------------------------
    # flatten diagonals
    # ------------------------------------------------------------------
    G_R_diag     = np.concatenate([np.diag(b) for b in G_R], dtype=complex)
    G_lesser_diag = np.concatenate([np.diag(b) for b in G_lesser], dtype=complex)

    return G_R_diag, G_lesser_diag

# ----------------------------------------------------------------------
# 3.  Benchmark driver
# ----------------------------------------------------------------------
def benchmark(num_blocks=500, block_size=2, energies=(0.01, 0.2, 0.5)):
    H, N, Sigma_left, Sigma_right, block_size = build_test_system(num_blocks, block_size)
    bs        = block_size
    N         = H.shape[0]

    for Ereal in energies:
        E = Ereal + 1e-12j          # retarded
        print(f"\nE = {Ereal:+.3f} eV  |  N = {N}")

        # ---------- direct inverse ----------
        t0 = time.perf_counter()
        A  = (E * np.eye(N) - H - Sigma_left - Sigma_right)
        G  = np.linalg.inv(A)
        diag_inv = np.diag(G)
        t1 = time.perf_counter()
        print(f"Direct inverse : {1e3*(t1-t0):8.2f} ms")
        

        # ---------- RGF ----------
        t0 = time.perf_counter()
        gR, gL = rgf_retarded_lesser(
            E, H,N, Sigma_left, Sigma_right, f_s = 1, f_d = 0, block_size=bs
        )
        t1 = time.perf_counter()
        print(f"RGF            : {1e3*(t1-t0):8.2f} ms")

        # validation
        maxerr = np.max(np.abs(diag_inv - gR))
        print(f"max |ΔGᵣ(ii)|   : {maxerr:8.2e}")

# ----------------------------------------------------------------------
if __name__ == "__main__":
    benchmark()



E = +0.010 eV  |  N = 1000
Direct inverse :   130.83 ms
RGF            :   142.75 ms
max |ΔGᵣ(ii)|   : 8.81e-11

E = +0.200 eV  |  N = 1000
Direct inverse :   106.15 ms
RGF            :   128.32 ms
max |ΔGᵣ(ii)|   : 3.13e-11

E = +0.500 eV  |  N = 1000
Direct inverse :   132.09 ms
RGF            :   145.52 ms
max |ΔGᵣ(ii)|   : 2.90e-08
